In [1]:
import pickle

from dataset import NewsDataset
from DistilbertModelMultilabel import DistilBertForSequenceClassification

from smooth_gradient import SmoothGradient
from integrated_gradient import IntegratedGradient

import torch
from torch import nn
from torch.utils.data import DataLoader
from transformers import DistilBertConfig, DistilBertTokenizer
import pandas as pd
from IPython.display import display, HTML
import numpy as np

In [2]:
config = DistilBertConfig()
tokenizer = DistilBertTokenizer.from_pretrained("distilbert-base-uncased")
# distilbert = DistilBertForSequenceClassification(config, num_labels=2)

In [3]:
# from transformers import DistilBertForSequenceClassification

In [4]:
config.multi=True

In [5]:
config.num_labels=10
distilbert = DistilBertForSequenceClassification.from_pretrained( "../tcav2/predict_sapsii/checkpoints/distilbert_OASIS__5.h5",from_tf=True,config=config)

All TF 2.0 model weights were used when initializing DistilBertForSequenceClassification.

All the weights of DistilBertForSequenceClassification were initialized from the TF 2.0 model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use DistilBertForSequenceClassification for predictions without further training.


In [6]:
# distilbert.state_dict

In [7]:
# import torch 
# torch.save(distilbert.state_dict(),'oasis_model.pt')

In [8]:
# distilbert = DistilBertForSequenceClassification(config=config)#.from_pretrained(None,config=config,state_dict='distilbert_cad_final1.pt')

In [9]:
criterion = nn.CrossEntropyLoss()

In [10]:
batch_size = 1

In [11]:
# "../predict_sapsii/checkpoints/distilbert_OASIS__5.h5"

In [12]:
# torch.load(path, map_location=torch.device('cpu'))

In [13]:
# path = './oasis_model.pt'

# if torch.cuda.is_available():
#     distilbert.load_state_dict(
#         torch.load(path)
#     )
# else:
#     distilbert.load_state_dict(
#         torch.load(path, map_location=torch.device('cpu'))
#     )
    
# # # with open('../label_encoder.sklrn', 'rb') as f:
# # #     le = pickle.load(f)

In [14]:
# distilbert = DistilBertForSequenceClassification.from_pretrained('../tcav/bertcheckpoints/distilbert/distilbert_cad_final1.h5',output_attentions=True, from_tf=True)

In [15]:
# np.array([[0]*512]).size()

In [16]:
# predictions=distilbert(input_ids=torch.tensor([[0]*512]),return_dict=True,output_hidden_states=True)

In [18]:
test_df = pd.read_csv('../tcav2/data/oasis_test.csv',nrows=2)

In [36]:
classes = list(test_df.columns[3:])

In [27]:
texts = list(test_df['TEXT'])

In [28]:
texts[0]

"MICU nursing admission Note\nPt found unresponsive this am. taken to ED with resp distress. Narcan, abx and nebs given. CXR showed PCP, ?SARS. Pt also states that she has bloody sputum, thus pt placed on airborne iso room. pt admitted to Cocaine and Heroin use last noc. arrived to MICU A slurring speech, drowsy. additional Narcan given. Pt alert and oriented. Arrived on Venturi mask, tolerating NC now at 3liters with sats 95-98%.\n\nResp- On 3 liters NC, no SOB at rest, does become dyspneic with anxiety. Reports cough X3 days. awaiting sputum spec, pt aware of need for sample. On nebs. Lung sounds clear with Occ wheezes.\n\nCV- NSR with HR 90-100. BP stable. without edema, clubbing. palpable peripheral pulses. Currently repleating K (40MEQ) IV.\n\nSkin- dry and intact. no breaks in skin integrity.\n\nNeuro- Alert and oriented, especially after 2nd does of Narcan. Moves all extermities, without loss of sensation. [**Last Name (un) 207**] and coop. pt c/o diffuse bone pain.\n\nGI/GU- bo

In [29]:
all_instances = []
# texts=['patient had high bp', 'Patient is 76 years old']
for text in texts:
    test_example = [
        [text], 
        [""]
    ]

    test_dataset = NewsDataset(
        data_list=test_example,
        tokenizer=tokenizer,
        max_length=config.max_position_embeddings, 
    )

    test_dataloader = DataLoader(
        test_dataset,
        batch_size=batch_size,
        shuffle=False,
    )

    integrated_grad = IntegratedGradient(
        distilbert, 
        criterion, 
        tokenizer, 
        show_progress=True,
        encoder="bert"
    )
    instances = integrated_grad.saliency_interpret(test_dataloader)
    all_instances.append(instances)

100%|██████████| 1/1 [00:33<00:00, 33.03s/it]


In [30]:
integrated_grad = IntegratedGradient(
        distilbert, 
        criterion, 
        tokenizer, 
        show_progress=True,
        encoder="bert"
    )

In [31]:
coloder_string = integrated_grad.colorize(instances[0])
display(HTML(coloder_string))

In [56]:
classes[7]

'urineoutput_score'

In [28]:
grads_df = pd.DataFrame([instance[0] for instance in all_instances])#.to_csv('all_test_grads.csv')

In [39]:
def unravel(lst):
    return [i for j in lst for i in j]

In [40]:
import pandas as pd

# grads_df = pd.read_csv('all_test_grads2.csv')
grads_df = pd.read_csv('all_train_grads_oasis.csv')

In [41]:
grads_df['tokens']=grads_df['tokens'].apply(eval)

In [42]:
grads_df['grad'] = grads_df['grad'].apply(eval)

In [43]:
grads_df.sort_values(by='prob',ascending=False)

,Unnamed: 0,tokens,grad,label,prob
184,184,"[[CLS], [, *, *, 210, ##3, -, 8, -, 15, *, *, ...","[0.007302305195480585, 0.006230922415852547, 0...",9,0.197702
2432,2432,"[[CLS], respiratory, care, pt, was, admitted, ...","[0.005153948906809092, 0.04265802353620529, 0....",9,0.195273
3597,3597,"[[CLS], respiratory, care, :, patient, receive...","[0.002436938928440213, 0.03851640224456787, 0....",9,0.194829
2815,2815,"[[CLS], cc, ##u, np, ##n, pt, arrived, from, c...","[0.001412274083122611, 0.012089354917407036, 0...",9,0.193610
1931,1931,"[[CLS], res, ##p, care, note, ,, pt, remains, ...","[0.0014866458950564265, 0.028132222592830658, ...",9,0.192721
...,...,...,...,...,...
5136,5136,"[[CLS], mr, [, *, *, known, last, ##name, 84, ...","[0.006509016267955303, 0.011597655713558197, 0...",6,0.123108
1605,1605,"[[CLS], pm, ##ic, ##u, np, ##n, 7, ##p, -, 7, ...","[0.0006316918297670782, 0.01141029130667448, 0...",6,0.122764
1220,1220,"[[CLS], [, *, *, 212, ##0, -, 10, -, 1, *, *, ...","[0.009816708974540234, 0.0065320380963385105, ...",1,0.122558
4540,4540,"[[CLS], [, *, *, 217, ##7, -, 8, -, 14, *, *, ...","[0.0017538964748382568, 0.010574903339147568, ...",7,0.122526


In [45]:
ex = grads_df.sort_values(by='prob',ascending=False).iloc[3]
coloder_string=integrated_grad.colorize({'tokens':ex['tokens'], 'grad':ex['grad'], 'label':ex['label'],'prob':ex['prob']})
display(HTML(coloder_string))

In [49]:
label = 1
print(classes[label])
all_class_grads = pd.DataFrame(  list(zip( unravel(grads_df[grads_df['label']==label]['tokens']),  unravel(grads_df[grads_df['label']==label]['grad']) )) , columns=['tokens', 'grad']  )

preiculos_score


In [50]:
word =''
score=0
words=[]
scores=[]
for i in reversed(range(len(all_class_grads))):
    token=all_class_grads['tokens'].iloc[i]
    if token.startswith('##'):
        word = token[2:]+word
        score = max(score,all_class_grads['grad'].iloc[i] )
#         print(word)
    else:
        word = token+word
#         print(word)
        words.append(word)
        score = max(score,all_class_grads['grad'].iloc[i] )
        scores.append(score)
        word=''
        score=0

In [51]:
tokens_grouped = pd.DataFrame(list(zip(words, scores)), columns=['words', 'grad']).groupby('words')
means = tokens_grouped['grad'].mean()
cts = tokens_grouped['grad'].count()
wrds = tokens_grouped.apply(lambda group: str(group.name))
class_df = pd.DataFrame(list(zip(wrds,means,cts)), columns=['word','grad_mean','word_count' ]).sort_values(by='grad_mean',ascending=False)
class_df = class_df[class_df['word_count']>5]
print(class_df.head(25).to_latex(index=False, float_format="%.3f"))

\begin{tabular}{lrr}
\toprule
            word &  grad\_mean &  word\_count \\
\midrule
             npn &      0.020 &          16 \\
        incision &      0.016 &           9 \\
     hemodynamic &      0.012 &           6 \\
             tpn &      0.012 &           6 \\
        infusing &      0.012 &          15 \\
    unremarkable &      0.012 &          13 \\
         tylenol &      0.012 &           7 \\
    namepattern1 &      0.011 &           7 \\
             ica &      0.011 &           6 \\
   opacification &      0.011 &           6 \\
               e &      0.011 &           7 \\
         carevue &      0.011 &          10 \\
             fhp &      0.011 &           9 \\
            ciwa &      0.010 &           7 \\
           bipap &      0.010 &           7 \\
     mediastinal &      0.010 &           7 \\
             med &      0.010 &          15 \\
 cardiopulmonary &      0.010 &           6 \\
             neg &      0.009 &          12 \\
             dka & 

In [38]:
word =''
score=0
negative_words=[]
negative_scores=[]
for i in reversed(range(len(all_negative_grads))):
    token=all_negative_grads['tokens'].iloc[i]
    if token.startswith('##'):
        word = token[2:]+word
        score = max(score,all_negative_grads['grad'].iloc[i] )
#         print(word)
    else:
        word = token+word
#         print(word)
        negative_words.append(word)
        score = max(score,all_negative_grads['grad'].iloc[i] )
        negative_scores.append(score)
        word=''
        score=0

In [39]:
tokens_grouped = pd.DataFrame(list(zip(negative_words, negative_scores)), columns=['words', 'grad']).groupby('words')
means = tokens_grouped['grad'].mean()
cts = tokens_grouped['grad'].count()
wrds = tokens_grouped.apply(lambda group: str(group.name))
negative_df = pd.DataFrame(list(zip(wrds,means,cts)), columns=['word','grad_mean','word_count' ]).sort_values(by='grad_mean',ascending=False)
negative_df = negative_df[negative_df['word_count']>5]
print(negative_df.head(25).to_latex(index=False, float_format="%.3f"))

\begin{tabular}{lrr}
\toprule
                word &  grad\_mean &  word\_count \\
\midrule
                 cad &      0.094 &          76 \\
               [SEP] &      0.081 &        6819 \\
                 lad &      0.075 &           7 \\
           narrative &      0.069 &          37 \\
                cabg &      0.068 &          10 \\
                 npn &      0.064 &         762 \\
           cadaveric &      0.057 &           7 \\
                 pvc &      0.047 &         480 \\
                pvcs &      0.042 &         166 \\
                 ccu &      0.040 &         306 \\
                simv &      0.037 &          72 \\
         prostectomy &      0.036 &           6 \\
         thorocotomy &      0.036 &           6 \\
              pedial &      0.035 &          20 \\
               tsicu &      0.035 &         516 \\
   gastrojejunostomy &      0.033 &           7 \\
           lobectomy &      0.032 &          65 \\
                sicu &      0.032 &      

In [93]:
list(means

['grad']

In [72]:
pd.DataFrame(list(zip(words, scores)), columns=['words', 'grad'])

,words,grad
0,',0.028707
1,a,0.051793
2,i,0.001401
3,s,0.000330
4,e,0.005617
...,...,...
117397,L,0.007958
117398,C,0.003365
117399,[,0.001415
117400,',0.005063


In [57]:
import pandas as pd
pd.DataFrame(list(zip(instances[0]['tokens'], instances[0]['grad'])))

IndexError: list index out of range

In [20]:
instances

[{'tokens': ['[CLS]',
   "'",
   'patient',
   'was',
   'a',
   'pleasant',
   'male',
   'in',
   'his',
   'early',
   '40',
   '##s',
   'who',
   'was',
   'given',
   'pl',
   '##avi',
   '##x',
   'and',
   'as',
   '##pi',
   '##rin',
   'for',
   'cad',
   'and',
   'required',
   'cab',
   '##g',
   "'",
   '[SEP]',
   'transform',
   '##er',
   '-',
   'based',
   'models',
   'have',
   'taken',
   'a',
   'leading',
   'role',
   'in',
   'nl',
   '##p',
   'today',
   '.',
   '[SEP]'],
  'grad': [0.0024040760472416878,
   0.011202486231923103,
   0.013759119436144829,
   0.002332319738343358,
   0.011373970657587051,
   0.02735648676753044,
   0.015535157173871994,
   0.030777039006352425,
   0.008332711644470692,
   0.013340604491531849,
   0.0001066805562004447,
   0.0027349493466317654,
   0.004292184952646494,
   0.00442542415112257,
   0.007150531280785799,
   0.0028611449524760246,
   0.015010221861302853,
   0.018782952800393105,
   0.01669878326356411,
   0.009789